Imports

In [ ]:
import pandas as pd
import torch

from src.models import LateFusionClassifier, IntermediateFusionClassifier
from src.training import train_task3

from src.datasets import AssessmentPairs, train_val_split, AssessmentTorchDataset
from torch.utils.data import DataLoader

pairs = AssessmentPairs("datasets").load_pairs()
train_pairs, val_pairs = train_val_split(pairs)

train_ds = AssessmentTorchDataset(train_pairs)
val_ds   = AssessmentTorchDataset(val_pairs)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)


Prepare Training

In [ ]:
batch = next(iter(train_loader))
rgb_in_ch = batch["rgb"].shape[1]
lidar_in_ch = batch["lidar"].shape[1]
print("rgb_in_ch:", rgb_in_ch, "lidar_in_ch:", lidar_in_ch)


Run all models

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
EPOCHS = 10
LR = 1e-3
EMB = 200

runs = []

m = LateFusionClassifier(rgb_in_ch, lidar_in_ch, emb_size=EMB, num_classes=2)
out = train_task3(m, train_loader, val_loader, device=device, epochs=EPOCHS, lr=LR)
runs.append({"model": "late_fusion", **{k: out[k] for k in ["val_loss_final","params","sec_per_epoch_avg","gpu_mem_mb_peak"]}})

for fusion in ["concat", "add", "hadamard"]:
    m = IntermediateFusionClassifier(fusion, rgb_in_ch, lidar_in_ch, emb_size=EMB, num_classes=2)
    out = train_task3(m, train_loader, val_loader, device=device, epochs=EPOCHS, lr=LR)
    runs.append({"model": f"intermediate_{fusion}", **{k: out[k] for k in ["val_loss_final","params","sec_per_epoch_avg","gpu_mem_mb_peak"]}})

df = pd.DataFrame(runs).sort_values("val_loss_final")
df


Export Results

In [ ]:
print(df.to_markdown(index=False))
